In [16]:
using StatsPlots, UnROOT, StatsBase, MPSensitivity, CategoricalArrays, Revise
using FHist, LaTeXStrings, MPThemes, DataFrames, DataFramesMeta, Distributions, Markdown

Custom functions are placed in the MiscFuncs.jl file.

In [2]:
include("MiscFuncs.jl")
using .MiscFuncs

In [5]:
plotlyjs()
my_vibrant(;
    size           = (800, 600),
    legend         = :outertopleft,
    guidefontsize  = 8,
    tickfontsize   = 8,
    titlefontsize  = 8,
    legendfontsize = 6,
    left_margin    = 4Plots.mm,
    right_margin   = 4Plots.mm,
    top_margin     = 4Plots.mm,
    bottom_margin  = 4Plots.mm,
    dpi            = 200,
);

In [14]:
baseDir = "/home/shoram/Work/PhD_Thesis/Job15/AngularCorrelations/"

"/home/shoram/Work/PhD_Thesis/Job15/AngularCorrelations/"

In [11]:
f = ROOTFile(
    baseDir*"AngularCorrelationAllEnergies96MilEvents.root",
);
tree = DataFrame(LazyTree(f, "tree", keys(f["tree"])));

### ``@transform`` adds a column ``:ESum`` to the ``tree`` which contains the sum of the electron energies

In [12]:
@transform! tree :ESum = :reconstructedEnergy2 + :reconstructedEnergy1;


### Initializing constants.

In [8]:

dEmitted = 1 # dθdif in degrees
nBins    = Int(180 / dEmitted)
minAngle = 0
maxAngle = 180
binWidth = maxAngle / nBins

minEnergy = 500
maxEnergy = 3500
dEnergy   = 500

xPts = minAngle:dEmitted:(maxAngle - dEmitted) # x-points for plotting


0:1:179

In [9]:
labelsEnergy = [
    string("E ∈ (", minE, ", ", minE + dEnergy, " )") for
    minE in minEnergy:dEnergy:(maxEnergy-dEnergy)
]

labelsAngles = [
    string("θ ∈ (", minA, ", ", minA + dEmitted, " )") for
    minA in minAngle:dEmitted:(maxAngle-dEmitted)
]

colors = [palette(:seaborn_bright)[i] for i in 1:length(palette(:seaborn_bright))];

In [17]:
h2d = histogram2d(
    tree.thetaEmitted,
    tree.thetaEscaped;
    nbins        = (nBins, nBins),
    xlabel       = "θemitted -> θ",
    ylabel       = "θescaped -> ϕ",
    legend       = :topright,
    title        = string("θesc vs θemit, ", nrow(tree), " entries"),
    lims         = (0, 180),
    aspect_ratio = 1,
)
savefig(h2d, joinpath(baseDir, string("h2d.png")))

### The individual stats (``mean, mode, median, variance, efficiency``) are calculated from the vertical slices of ``h2d`` where a slice of ``dθ``. The output is saved in a ``df_stats`` DataFrame which contains the following fiels:
1. "minThetaEmitted"
2. "maxThetaEmitted"
3. "minE"
4. "maxE"
5. "mean"
6. "mode"
7. "median"
8. "variance"
9. "escEfficiency" 

In [18]:
df_stats = DataFrame(;
    minThetaEmitted = Real[],
    maxThetaEmitted = Real[],
    minE = Real[],
    maxE = Real[],
    mean = Real[],
    mode = Real[],
    median = Real[],
    variance = Real[],
    escEfficiency = Real[],
)


,minThetaEmitted,maxThetaEmitted,minE,maxE,mean,mode,median,variance,escEfficiency
,Real,Real,Real,Real,Real,Real,Real,Real,Real


In [19]:
for minThetaEmitted in minAngle:dEmitted:(maxAngle - dEmitted)  # for loop over emitted slices, the size of slize is determined by dEmitted 
    maxThetaEmitted = minThetaEmitted + dEmitted

    for minE in minEnergy:dEnergy:(maxEnergy - dEnergy)     # for loop over energy slices of the sum of energies 
        maxE = minE + dEnergy

        push!(
            df_stats,
            get_slice_stats(minThetaEmitted, maxThetaEmitted, minE, maxE, tree, binWidth),
        )
    end
end